## Demo file on training the auto-encoder model

In [1]:
import numpy as np

In [2]:
sample = np.load("trim_sample.npy",allow_pickle=True)

In [3]:
sample.shape

(4325, 30000)

In [4]:
train = sample[:100,]
test  =  sample[2000:2100,]

In [5]:
train.shape

(100, 30000)

In [6]:
from autoencode import Auto_Encoder
import torch
import torchvision
from torch import nn, optim
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
import os

In [7]:
learning_rate = 0.001
batch_size = 64
epochsize = 30

In [8]:
AE = Auto_Encoder()
MSELoss = nn.MSELoss()
optimizer = optim.Adam(AE.parameters(), lr=learning_rate)

In [9]:
AE.parameters()

<generator object Module.parameters at 0x162336a40>

In [10]:
Train = torch.from_numpy(train).to(torch.float32)
Train.shape

torch.Size([100, 30000])

In [11]:
loss1 = MSELoss (AE(Train[0]),Train[0])

In [12]:
loss1

tensor(0.1226, grad_fn=<MseLossBackward0>)

In [13]:
### Train
for s in Train:
    s_prime = AE(s)

    # loss 
    loss = MSELoss(s_prime, s)
    #print(loss)
    # update
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [14]:
loss2 = MSELoss (AE(Train[0]),Train[0])

In [15]:
loss2

tensor(264.2661, grad_fn=<MseLossBackward0>)

In [16]:
loss1 - loss2

tensor(-264.1434, grad_fn=<SubBackward0>)

In [17]:
### testing
losses = []
Test = torch.from_numpy(test).to(torch.float32)
for s in Test:
    s_prime = AE(s)

    # loss 
    loss = MSELoss(s_prime, s)
    losses.append((loss.detach().numpy()))
    #print(loss)
    # update
    

In [18]:
np.mean(losses)

5.21315

In [19]:
sample1 = Test[0]
sample1 = sample1.reshape(-1,3)

In [20]:
recover_sample = AE(Test[0]).reshape(-1,3)

In [21]:
import open3d as o3d

In [22]:
pcd1 = o3d.geometry.PointCloud()
pcd2 = o3d.geometry.PointCloud()
pcd1.points = o3d.utility.Vector3dVector(sample1.detach().numpy()) 
pcd2.points = o3d.utility.Vector3dVector(recover_sample.detach().numpy() )

In [23]:
o3d.visualization.draw_geometries([pcd1])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [24]:
o3d.visualization.draw_geometries([pcd2])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [25]:
np.sum(recover_sample.detach().numpy() != 0)

30000

In [26]:
recover_sample[recover_sample!=0]

tensor([-4.8422, -2.9470,  3.8767,  ...,  1.2082, -3.5098,  4.1932],
       grad_fn=<IndexBackward0>)